# Replication of Regression from Inflation Expectation Paper

## Data

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [2]:
#Read in data
df=pd.read_csv("data/data.csv")

In [3]:
#Period between 1984:1 to 2012:12
cleaned_data=df[(df["yyyymm"] >= 198401)&(df["yyyymm"] <= 201212)]
#Only keep first time responces
cleaned_data=cleaned_data[cleaned_data["datepr"].isnull()]

In [4]:
#Extra Transforms

#Age
cleaned_data["AGE2"] = cleaned_data["AGE"]*cleaned_data["AGE"]
cleaned_data["AGE3"] = cleaned_data["AGE2"]*cleaned_data["AGE"]

#Inflation DLZ
cleaned_data["PX1_ZLB"] = cleaned_data["PX1"]*cleaned_data["ZLB"]

trans = ["AGE2","AGE3","PX1_ZLB"]

In [5]:
#Inflation Expectation
pi=["PX1","PX1_ZLB"]

In [6]:
#Control for demographics

#Age cubed left out for convergence issues. Has almost 0 effect
control = ["SEX","MARRY","ECLGRD","AFRICAN","HISPANIC","NATIVE","ASIAN","WEST",
       "NORTHEAST","SOUTH","FAMSIZE","AGE","AGE2","INCOME"]

In [7]:
#other controls
ocontrol = ["ZLB","PEXP","RINC","RATEX","BUS12","BUS5","UNEMP","PAGO","GOVT",
            "BUS12AG","PX1DISP","VXO","FFR","UNRATE","INFLATION","INFLVOLA","CPIDURABLES"]

In [8]:
#Indep Variables
x = cleaned_data[pi+ocontrol+control]

In [9]:
#Dependent Variable
y = cleaned_data["DUR"]

## Probit Model

In [10]:
model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs', maxiter = 500000)
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.624525
         Iterations: 108
         Function evaluations: 118
         Gradient evaluations: 118
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -42265.
Model:                   OrderedModel   AIC:                         8.460e+04
Method:            Maximum Likelihood   BIC:                         8.492e+04
Date:                Fri, 06 May 2022                                         
Time:                        13:22:10                                         
No. Observations:               67675                                         
Df Residuals:                   67640                                         
Df Model:                          35                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------

In [11]:
num_of_thresholds = 2
model.transform_threshold_params(res.params[-num_of_thresholds:])

array([       -inf, -1.46981365, -1.34185356,         inf])

# Robustness Checks

## Excluding Idiosyncratic Expecttions Controls

In [12]:
model_data = pd.concat([x,y],axis=1)

In [13]:
#Excluding Policy Trust
model_data = model_data.drop(["GOVT"],axis=1)
y = model_data["DUR"]
x = model_data.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.626448
         Iterations: 101
         Function evaluations: 109
         Gradient evaluations: 109


In [14]:
#Excluding All idiosyncratic variables
model_data = model_data.drop(["PEXP","RINC","RATEX","BUS12","BUS5","UNEMP"],axis=1)
y = model_data["DUR"]
x = model_data.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.651363
         Iterations: 93
         Function evaluations: 100
         Gradient evaluations: 100


In [15]:
#Reset
x = cleaned_data[pi+ocontrol+control]
y = cleaned_data["DUR"]
model_data = pd.concat([x,y],axis=1)

In [16]:
# With Gas Price Expectations, Home Ownership and sub prob
model_data = pd.concat([cleaned_data[["GAS1","HOMEOWN","PINC2","PJOB"]],model_data], axis=1)
model_data = model_data[cleaned_data["yyyymm"] >= 199801]

y = model_data["DUR"]
x = model_data.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.677328
         Iterations: 109
         Function evaluations: 116
         Gradient evaluations: 116


In [17]:
#Reset
x = cleaned_data[pi+ocontrol+control]
y = cleaned_data["DUR"]
model_data = pd.concat([x,y],axis=1)

In [18]:
#5 year inflation expectations instead of one year
model_data = model_data[cleaned_data["yyyymm"] >= 199004]


y = model_data["DUR"]
x = model_data.drop(["DUR"],axis=1)


x = x.drop(["PX1","PX1_ZLB"],axis = 1)
x["PX5"] = cleaned_data["PX5"]
x["PX5_ZLB"] = x["PX5"] * x["ZLB"]

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.646885
         Iterations: 98
         Function evaluations: 106
         Gradient evaluations: 106


## Cross Sectional Heterogenity

In [19]:
#Reset
x = cleaned_data[pi+ocontrol+control]
y = cleaned_data["DUR"]
model_data = pd.concat([x,y],axis=1)

In [20]:
#Old
model_old = model_data[model_data["AGE"] > 48]

y = model_old["DUR"]
x = model_old.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.656675
         Iterations: 112
         Function evaluations: 122
         Gradient evaluations: 122


In [21]:
#Young
model_young = model_data[model_data["AGE"] < 48]

y = model_young["DUR"]
x = model_young.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.599970
         Iterations: 91
         Function evaluations: 98
         Gradient evaluations: 98


In [22]:
#College
model_college = model_data[model_data["ECLGRD"] == 1]
model_college = model_college.drop(["ECLGRD"],axis = 1)

y = model_college["DUR"]
x = model_college.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.627368
         Iterations: 94
         Function evaluations: 104
         Gradient evaluations: 104


In [23]:
#No College
model_nocollege = model_data[model_data["ECLGRD"] == 0]
model_nocollege = model_nocollege.drop(["ECLGRD"],axis = 1)

y = model_nocollege["DUR"]
x = model_nocollege.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.621165
         Iterations: 98
         Function evaluations: 107
         Gradient evaluations: 107


In [24]:
#Top 20% income
model_top = model_data[cleaned_data["income"] >= cleaned_data["income"].quantile(q=0.8)]

y = model_top["DUR"]
x = model_top.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.636086
         Iterations: 96
         Function evaluations: 105
         Gradient evaluations: 105


In [25]:
#Bottom 20% income
model_bottom = model_data[cleaned_data["income"] <= cleaned_data["income"].quantile(q=0.2)]

y = model_bottom["DUR"]
x = model_bottom.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
#print(res.summary())

Optimization terminated successfully.
         Current function value: 0.640264
         Iterations: 108
         Function evaluations: 113
         Gradient evaluations: 113


# Further Robustness Tests

## Residuals

In [26]:
#Reset
x = cleaned_data[pi+ocontrol+control]
y = cleaned_data["DUR"]
model_data = pd.concat([x,y],axis=1)

In [ ]:
#Model
model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')

Optimization terminated successfully.
         Current function value: 0.624525
         Iterations: 108
         Function evaluations: 118
         Gradient evaluations: 118


In [ ]:
#Predict
y_hat = res.predict(x)

In [ ]:
#Get avalible residuals
y_hat = y_hat.dropna()
y_real = y[y_hat.index]
y_res = np.array((y_real+1))-np.array(y_hat).argmax(1)
y_res = y_res[~np.isnan(y_res)]
y_res = pd.DataFrame(y_res)

In [ ]:
#Note negative bias
y_res.value_counts()

## Check OLS Fitting

In [ ]:
#Reset
x = cleaned_data[pi+ocontrol+control]
y = cleaned_data["DUR"]
model_data = pd.concat([x,y],axis=1)

In [ ]:
mod = sm.OLS(y, x, missing="drop")
test=mod.fit()

In [ ]:
test.summary()